In [1]:
import flash_attn
import torch
import torch.nn.functional as F
from einops import rearrange

# Without mask ✅

In [2]:
batch, n_heads, seq_len, d_head = 1,1,2,2
inputs = torch.randn(batch, n_heads, seq_len, d_head, dtype=torch.float16, device="cuda")

query = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)
key = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)
value = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)

In [3]:
F.scaled_dot_product_attention(query=inputs, key=inputs, value=inputs)

tensor([[[[-0.1075,  2.6523],
          [-0.6958,  2.2676]]]], device='cuda:0', dtype=torch.float16)

In [4]:
cu_seqlens = torch.tensor([0,2], dtype=torch.int32, device="cuda")
max_seqlen = 2

flash_attn.flash_attn_varlen_func(
    query,
    key,
    value,
    cu_seqlens_q=cu_seqlens,
    cu_seqlens_k=cu_seqlens,
    max_seqlen_q=max_seqlen,
    max_seqlen_k=max_seqlen,
)

tensor([[[-0.1075,  2.6523]],

        [[-0.6958,  2.2676]]], device='cuda:0', dtype=torch.float16)

# With mask

In [42]:
batch, n_heads, seq_len, d_head = 1,8,3,2
inputs = torch.randn(batch, n_heads, seq_len, d_head, dtype=torch.float16, device="cuda")

query = rearrange(
    inputs, "batch n_heads seq_len d_head -> (seq_len batch) n_heads d_head"
)
key = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)
value = rearrange(
    inputs, "batch n_heads seq_len d_head -> (seq_len batch) n_heads d_head"
)

In [43]:
inputs

tensor([[[[ 0.7280,  0.5229],
          [ 0.2413, -1.3203],
          [-1.2139, -0.8760]],

         [[-0.3494,  0.0082],
          [-1.7969,  0.1509],
          [-0.0773,  0.4136]],

         [[-0.2147,  2.4062],
          [ 0.5996, -1.8535],
          [ 1.0215, -0.4619]],

         [[-0.0689,  0.2925],
          [ 0.9292,  1.3545],
          [-0.0526, -0.9819]],

         [[-2.0996, -0.9917],
          [-2.4336, -0.9766],
          [ 1.4688, -0.3450]],

         [[-1.3271, -0.4514],
          [-0.6494, -0.3589],
          [-0.6704, -1.6143]],

         [[-0.0991,  0.1261],
          [ 0.6064,  0.1158],
          [-0.7236,  0.2693]],

         [[ 1.2568, -0.8354],
          [ 0.8306, -0.1301],
          [ 0.3018, -0.3230]]]], device='cuda:0', dtype=torch.float16)

In [44]:
attention_mask = torch.tensor([[True, False, False], [True, True, True], [True, True, False]], device="cuda")
F.scaled_dot_product_attention(query=inputs, key=inputs, value=inputs, attn_mask=attention_mask)

tensor([[[[ 0.7280,  0.5229],
          [-0.1418, -0.9766],
          [ 0.3259, -1.0000]],

         [[-0.3494,  0.0082],
          [-1.4629,  0.1572],
          [-1.1172,  0.0839]],

         [[-0.2147,  2.4062],
          [ 0.6660, -1.6191],
          [ 0.5010, -1.3369]],

         [[-0.0689,  0.2925],
          [ 0.7344,  1.0889],
          [ 0.2461,  0.6279]],

         [[-2.0996, -0.9917],
          [-2.3105, -0.9819],
          [-2.2383, -0.9854]],

         [[-1.3271, -0.4514],
          [-0.9072, -0.8535],
          [-1.0596, -0.4148]],

         [[-0.0991,  0.1261],
          [ 0.0513,  0.1571],
          [ 0.1904,  0.1219]],

         [[ 1.2568, -0.8354],
          [ 0.8911, -0.4863],
          [ 1.0703, -0.5269]]]], device='cuda:0', dtype=torch.float16)

In [45]:
cu_seqlens_q = torch.tensor([0, 1, 4, 6], dtype=torch.int32, device="cuda")
cu_seqlens_k = torch.tensor([0, 1, 4, 6], dtype=torch.int32, device="cuda")
max_seqlen = 3

flash_attn.flash_attn_varlen_func(
    query,
    key,
    value,
    cu_seqlens_q=cu_seqlens_q,
    cu_seqlens_k=cu_seqlens_k,
    max_seqlen_q=max_seqlen,
    max_seqlen_k=max_seqlen,
)

tensor([[[ 0.7280,  0.5229],
         [-0.3494,  0.0082],
         [-0.2147,  2.4062],
         [-0.0689,  0.2925],
         [-2.0996, -0.9917],
         [-1.3271, -0.4514],
         [-0.0991,  0.1261],
         [ 1.2568, -0.8354]],

        [[-0.2139, -0.9868],
         [-1.4854,  0.1635],
         [ 0.6318, -1.5400],
         [ 0.7690,  1.0791],
         [-2.4121, -0.9688],
         [-0.5156, -0.8481],
         [ 0.0823,  0.1155],
         [ 0.4487, -0.1577]],

        [[-0.7095, -0.8687],
         [-0.6572,  0.1956],
         [ 0.6680, -1.0166],
         [ 0.0734, -0.4270],
         [ 1.1631, -0.2988],
         [-0.6094, -1.2568],
         [-0.1981,  0.1519],
         [ 0.4050, -0.1572]]], device='cuda:0', dtype=torch.float16)

# Another example

In [20]:
batch, n_heads, seq_len, d_head = 3,1,5,1
inputs = torch.randn(batch, n_heads, seq_len, d_head, dtype=torch.float16, device="cuda")
inputs

tensor([[[[-1.1211],
          [-0.7690],
          [ 0.5044],
          [ 1.0645],
          [ 0.1743]]],


        [[[ 1.4844],
          [ 0.8862],
          [ 0.8584],
          [ 0.5596],
          [-2.2617]]],


        [[[ 0.7026],
          [ 0.2722],
          [ 1.6641],
          [-0.1707],
          [ 0.8115]]]], device='cuda:0', dtype=torch.float16)

In [32]:
attention_mask1 = torch.tensor([
    [1,1,1,0,0],
    [1,1,0,0,0],
    [1,1,1,1,1],
],device="cuda", dtype=torch.bool)
attention_mask1 = rearrange(
    attention_mask1, "batch seq_len -> batch 1 seq_len 1"
)

In [33]:
attention_mask2 = torch.tensor([
    [[1,1,1,0,0], [1,1,1,0,0], [1,1,1,0,0], [1,1,1,0,0], [1,1,1,0,0]],
    [[1,1,0,0,0], [1,1,0,0,0], [1,1,0,0,0], [1,1,0,0,0], [1,1,0,0,0]],
    [[1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1]]
],device="cuda", dtype=torch.bool).unsqueeze(1)

In [34]:
attention_mask3 = torch.tensor([
    [[1,1,1,0,0], [1,1,1,0,0], [1,1,1,0,0], [0,0,0,0,0], [0,0,0,0,0]],
    [[1,1,0,0,0], [1,1,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]],
    [[1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1], [1,1,1,1,1]]
],device="cuda", dtype=torch.bool).unsqueeze(1)

In [35]:
F.scaled_dot_product_attention(query=inputs, key=inputs, value=inputs, attn_mask=attention_mask1)

tensor([[[[-0.6611],
          [-0.4995],
          [ 0.2817],
          [    nan],
          [    nan]]],


        [[[ 1.1250],
          [ 1.0088],
          [    nan],
          [    nan],
          [    nan]]],


        [[[ 0.9307],
          [ 0.7607],
          [ 1.2715],
          [ 0.5933],
          [ 0.9736]]]], device='cuda:0', dtype=torch.float16)

In [36]:
F.scaled_dot_product_attention(query=inputs, key=inputs, value=inputs, attn_mask=attention_mask2)

tensor([[[[-0.8486],
          [-0.7627],
          [-0.2006],
          [ 0.0750],
          [-0.3743]]],


        [[[ 1.3096],
          [ 1.2627],
          [ 1.2598],
          [ 1.2354],
          [ 1.0088]]],


        [[[ 0.9307],
          [ 0.7607],
          [ 1.2715],
          [ 0.5933],
          [ 0.9736]]]], device='cuda:0', dtype=torch.float16)

In [37]:
F.scaled_dot_product_attention(query=inputs, key=inputs, value=inputs, attn_mask=attention_mask3)

tensor([[[[-0.8486],
          [-0.7627],
          [-0.2006],
          [    nan],
          [    nan]]],


        [[[ 1.3096],
          [ 1.2627],
          [    nan],
          [    nan],
          [    nan]]],


        [[[ 0.9307],
          [ 0.7607],
          [ 1.2715],
          [ 0.5933],
          [ 0.9736]]]], device='cuda:0', dtype=torch.float16)

In [38]:
query = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)
key = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)
value = rearrange(
    inputs, "batch n_heads seq_len d_head -> (batch seq_len) n_heads d_head"
)

In [44]:
# indices=[ 0,  1,  2,  5,  6, 10, 11, 12, 13, 14]
cu_seqlens_q = torch.tensor([ 0, 3, 5, 10], dtype=torch.int32, device="cuda")
cu_seqlens_k = torch.tensor([ 0, 3, 5, 10], dtype=torch.int32, device="cuda")
max_seqlen_in_batch = 5

flash_attn.flash_attn_varlen_func(
    q=query,
    k=key,
    v=value,
    cu_seqlens_q=cu_seqlens_q,
    cu_seqlens_k=cu_seqlens_k,
    max_seqlen_q=max_seqlen_in_batch,
    max_seqlen_k=max_seqlen_in_batch,
)

tensor([[[-0.8486]],

        [[-0.7627]],

        [[-0.2002]],

        [[ 0.8159]],

        [[ 0.6538]],

        [[ 1.1250]],

        [[ 1.0088]],

        [[ 1.0010]],

        [[ 0.8853]],

        [[-2.2500]],

        [[ 0.0000]],

        [[ 0.0000]],

        [[ 0.0000]],

        [[ 0.0000]],

        [[ 0.0000]]], device='cuda:0', dtype=torch.float16)